<a href="https://colab.research.google.com/github/hwangho-kim/Utility-OAC/blob/main/Dash_%EA%B8%B0%EB%B0%98_%EB%8C%80%ED%99%94%ED%98%95_%EC%8A%B5%EA%B3%B5%EA%B8%B0%EC%84%A0%EB%8F%84_%EC%95%B1_(Jupyter_Lab_%EC%8B%A4%ED%96%89_%EA%B0%80%EB%8A%A5).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import dash
from dash import dcc, html, Input, Output, State, no_update
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import os
import io
import traceback

# 1. 기본 상수 정의
C_PA = 1.006  # 건공기의 정압비열 (kJ/kg°C)
C_PW = 1.86   # 수증기의 정압비열 (kJ/kg°C)
H_FG0 = 2501  # 0°C에서의 물의 증발잠열 (kJ/kg)
P_ATM = 101.325 # 표준 대기압 (kPa)
DATA_DIRECTORY = "./24Y/" # 고정된 데이터 디렉토리

# 2. 습공기 상태량 계산 함수 (이전과 동일)
def get_saturation_vapor_pressure_kPa(temp_c):
    if pd.isna(temp_c) or temp_c < -60: return np.nan
    return 0.61094 * np.exp((17.625 * temp_c) / (temp_c + 243.04))

def get_actual_vapor_pressure_kPa(temp_c, rh_percent):
    if pd.isna(temp_c) or pd.isna(rh_percent): return np.nan
    p_ws = get_saturation_vapor_pressure_kPa(temp_c)
    if pd.isna(p_ws): return np.nan
    return (rh_percent / 100.0) * p_ws

def get_humidity_ratio_kg_kg(temp_c, rh_percent, p_atm_kPa=P_ATM):
    if pd.isna(temp_c) or pd.isna(rh_percent): return np.nan
    p_w = get_actual_vapor_pressure_kPa(temp_c, rh_percent)
    if pd.isna(p_w): return np.nan
    denominator = p_atm_kPa - p_w
    if denominator <= 1e-9:
        p_ws_check = get_saturation_vapor_pressure_kPa(temp_c)
        if not pd.isna(p_ws_check) and abs(p_w - p_ws_check) < 1e-6 and rh_percent > 99.9:
             p_w_adjusted = p_ws_check * 0.99999
             denominator_adjusted = p_atm_kPa - p_w_adjusted
             if denominator_adjusted <= 1e-9: return float('inf')
             return 0.621945 * p_w_adjusted / denominator_adjusted
        return float('inf')
    return 0.621945 * p_w / denominator

def get_enthalpy_kJ_kg(temp_c, rh_percent, p_atm_kPa=P_ATM):
    if pd.isna(temp_c) or pd.isna(rh_percent): return np.nan
    w = get_humidity_ratio_kg_kg(temp_c, rh_percent, p_atm_kPa)
    if pd.isna(w) or w == float('inf'): return np.nan
    return C_PA * temp_c + w * (H_FG0 + C_PW * temp_c)

def get_temp_from_enthalpy_humidity_ratio(h_kJ_kg, w_kg_kg):
    if pd.isna(h_kJ_kg) or pd.isna(w_kg_kg): return np.nan
    denominator = C_PA + w_kg_kg * C_PW
    if abs(denominator) < 1e-9: return float('nan')
    return (h_kJ_kg - w_kg_kg * H_FG0) / denominator

# 3. Plotly용 습공기선도 배경 그리기 함수
def create_psychrometric_background_figure(temp_range_c, w_range_kg_kg):
    fig = go.Figure() # 새 Figure 객체 생성
    temps_c_plot = np.linspace(temp_range_c[0], temp_range_c[1], 100)

    w_saturated = np.array([get_humidity_ratio_kg_kg(t, 100) for t in temps_c_plot])
    valid_sat_indices = ~np.isnan(w_saturated) & ~np.isinf(w_saturated) & (w_saturated <= w_range_kg_kg[1]) & (w_saturated >= w_range_kg_kg[0])
    fig.add_trace(go.Scatter(
        x=temps_c_plot[valid_sat_indices], y=w_saturated[valid_sat_indices],
        mode='lines', name='상대습도 100%', line=dict(color='black', width=2),
        hoverinfo='skip', showlegend=True, legendgroup="background_lines"
    ))

    rh_curves_to_plot = [20, 40, 60, 80]
    for rh_val in rh_curves_to_plot:
        w_rh_plot = np.array([get_humidity_ratio_kg_kg(t, rh_val) for t in temps_c_plot])
        valid_rh_indices = ~np.isnan(w_rh_plot) & ~np.isinf(w_rh_plot) & (w_rh_plot <= w_range_kg_kg[1]) & (w_rh_plot >= w_range_kg_kg[0])
        if np.any(valid_rh_indices):
            fig.add_trace(go.Scatter(
                x=temps_c_plot[valid_rh_indices], y=w_rh_plot[valid_rh_indices],
                mode='lines', name=f'상대습도 {rh_val}%', line=dict(color='rgba(0,0,255,0.6)', width=1, dash='dash'),
                hoverinfo='skip', showlegend=True, legendgroup="background_lines"
            ))

    h_min_approx = get_enthalpy_kJ_kg(temp_range_c[0], 20 if temp_range_c[0] > 0 else 80)
    h_max_approx = get_enthalpy_kJ_kg(temp_range_c[1], 100)
    if pd.isna(h_min_approx) or h_min_approx == float('inf'): h_min_approx = 0
    if pd.isna(h_max_approx) or h_max_approx == float('inf'):
        h_max_approx = get_enthalpy_kJ_kg(temp_range_c[1], 80, P_ATM)
        if pd.isna(h_max_approx) or h_max_approx == float('inf'): h_max_approx = 120

    enthalpy_lines_values = np.unique(np.linspace(round(max(0,h_min_approx) / 10) * 10, round(h_max_approx / 10) * 10, 10).astype(int))
    enthalpy_lines_values = [h for h in enthalpy_lines_values if not pd.isna(h) and h != float('inf') and h != float('-inf') and h >=0]

    for h_val in enthalpy_lines_values:
        w_for_h_line = np.linspace(w_range_kg_kg[0], w_range_kg_kg[1], 50)
        temps_for_h_line = np.array([get_temp_from_enthalpy_humidity_ratio(h_val, w) for w in w_for_h_line])
        valid_h_points_indices = ~np.isnan(temps_for_h_line) & \
                                 (temps_for_h_line >= temp_range_c[0]) & (temps_for_h_line <= temp_range_c[1]) & \
                                 (w_for_h_line >= w_range_kg_kg[0]) & (w_for_h_line <= w_range_kg_kg[1])
        if np.sum(valid_h_points_indices) > 1:
            temps_plot_h = temps_for_h_line[valid_h_points_indices]
            w_plot_h = w_for_h_line[valid_h_points_indices]
            fig.add_trace(go.Scatter(
                x=temps_plot_h, y=w_plot_h, mode='lines',
                name=f'엔탈피={h_val}kJ/kg', line=dict(color='rgba(0,128,0,0.6)', width=1, dash='dot'),
                hoverinfo='skip', showlegend=False, legendgroup="background_lines"
            ))
            if len(temps_plot_h) > 0:
                idx_annot = len(temps_plot_h) // 2
                text_x, text_y = temps_plot_h[idx_annot], w_plot_h[idx_annot]
                if temp_range_c[0] <= text_x <= temp_range_c[1] and \
                   w_range_kg_kg[0] <= text_y <= w_range_kg_kg[1]:
                    fig.add_annotation(x=text_x, y=text_y, text=f"{h_val}", showarrow=False,
                                       font=dict(color="green", size=8), xanchor="center", yanchor="middle",
                                       bgcolor="rgba(255,255,255,0.5)", borderpad=2, textangle=-30)
    return fig # Figure 객체 반환

# --- 파일 및 컬럼 처리 함수 ---
STAGE_MAPPING_DEFINITIONS = [
    {'id': 'PHC', 'display_name': "1. PH/C (예열)", 'stage_keywords': ["PH/C", "PREHEAT", "예열"], 'temp_keyword': "온도", 'rh_keyword': "습도", 'color': '#ff7f0e'},
    {'id': 'PCC', 'display_name': "2. PC/C (예냉)", 'stage_keywords': ["PC/C", "PRECOOL", "예냉"], 'temp_keyword': "온도", 'rh_keyword': "습도", 'color': '#2ca02c'},
    {'id': 'CC',  'display_name': "3. C/C (냉각)", 'stage_keywords': ["C/C", "COOLINGCOIL", "냉각코일", "COOLING", "COOL"], 'temp_keyword': "온도", 'rh_keyword': "습도", 'color': '#d62728'},
    {'id': 'HC',  'display_name': "4. H/C (가열/재열)", 'stage_keywords': ["H/C", "HEATINGCOIL", "재열코일", "승온", "HEATING", "HEAT", "REHEAT"], 'temp_keyword': "온도", 'rh_keyword': "습도", 'color': '#9467bd'}
]
def map_columns_from_csv(df_columns):
    mapped_stages_config = []
    available_columns = list(df_columns)
    for stage_def in STAGE_MAPPING_DEFINITIONS:
        found_temp_col, found_rh_col = None, None
        for col_name in available_columns:
            col_lower = col_name.lower()
            is_stage_match = any(sk.lower() in col_lower for sk in stage_def['stage_keywords'])
            if is_stage_match:
                if stage_def['temp_keyword'].lower() in col_lower:
                    if found_temp_col is None: found_temp_col = col_name
                if stage_def['rh_keyword'].lower() in col_lower and col_name != found_temp_col:
                    if found_rh_col is None: found_rh_col = col_name
        if found_temp_col and found_rh_col:
            mapped_stages_config.append({
                'name': stage_def['display_name'], 'temp_col': found_temp_col,
                'rh_col': found_rh_col, 'color': stage_def['color']})
            if found_temp_col in available_columns: available_columns.remove(found_temp_col)
            if found_rh_col in available_columns: available_columns.remove(found_rh_col)
    return mapped_stages_config

# --- Dash 앱 설정 ---
# JupyterLab 환경에서 실행하기 위해 JupyterDash 사용 고려
try:
    from jupyter_dash import JupyterDash
    app = JupyterDash(__name__)
except ImportError:
    print("JupyterDash를 찾을 수 없습니다. 일반 Dash를 사용합니다. JupyterLab에서 실행하려면 'pip install jupyter-dash'를 실행하세요.")
    app = dash.Dash(__name__)

app.title = "대화형 습공기선도"

# 앱 레이아웃
app.layout = html.Div([
    html.H1("대화형 습공기선도 분석기", style={'textAlign': 'center'}),
    html.Div([
        html.Div([
            html.Label("CSV 파일 선택 (.oac.csv):"),
            dcc.Dropdown(id='file-dropdown', placeholder="파일을 선택하세요..."),
            html.Button('파일 목록 새로고침', id='refresh-files-button', n_clicks=0, style={'marginTop': '10px'}),
            html.Div(id='file-load-status', style={'marginTop': '10px', 'whiteSpace': 'pre-wrap'}) # 상태 메시지 줄바꿈 처리
        ], style={'width': '30%', 'display': 'inline-block', 'verticalAlign': 'top', 'padding': '20px'}),

        html.Div([
            dcc.Loading( # 로딩 스피너 추가
                id="loading-chart",
                type="circle", # 또는 "default", "cube", "dot"
                children=dcc.Graph(id='psychrometric-chart', config={'displayModeBar': True})
            ),
            dcc.Slider(id='time-slider', min=0, max=0, step=1, value=0, marks=None,
                       tooltip={"placement": "bottom", "always_visible": False},
                       disabled=True) # 초기에는 비활성화
        ], style={'width': '68%', 'display': 'inline-block', 'padding': '10px'})
    ]),
    dcc.Store(id='dataframe-store'),
    dcc.Store(id='mapped-stages-store'),
    dcc.Store(id='chart-ranges-store'),
    dcc.Store(id='static-traces-store') # 정적 배경 트레이스 저장용
])

# --- 콜백 함수들 ---
@app.callback(
    Output('file-dropdown', 'options'),
    Input('refresh-files-button', 'n_clicks')
)
def update_file_list(n_clicks):
    options = []
    status_message = ""
    if os.path.isdir(DATA_DIRECTORY):
        try:
            files = sorted([f for f in os.listdir(DATA_DIRECTORY) if "oac" in f.lower() and f.endswith(".csv")])
            options = [{'label': f, 'value': os.path.join(DATA_DIRECTORY, f)} for f in files]
            if not options:
                status_message = f"경고: '{DATA_DIRECTORY}'에서 'oac' CSV 파일을 찾을 수 없습니다."
            else:
                status_message = f"'{DATA_DIRECTORY}'에서 {len(options)}개의 파일 발견."
        except Exception as e:
            status_message = f"파일 목록 로드 중 오류: {e}"
    else:
        status_message = f"오류: 데이터 디렉토리 '{DATA_DIRECTORY}'를 찾을 수 없습니다. 생성 후 파일을 넣어주세요."
    print(status_message) # 콘솔에도 출력
    return options

@app.callback(
    [Output('dataframe-store', 'data'),
     Output('mapped-stages-store', 'data'),
     Output('chart-ranges-store', 'data'),
     Output('static-traces-store', 'data'), # 정적 트레이스 저장
     Output('time-slider', 'max'),
     Output('time-slider', 'marks'),
     Output('time-slider', 'value'),
     Output('time-slider', 'disabled'), # 슬라이더 활성화/비활성화
     Output('psychrometric-chart', 'figure'),
     Output('file-load-status', 'children')],
    Input('file-dropdown', 'value')
)
def load_and_prepare_data(selected_filepath):
    ctx = dash.callback_context
    if not selected_filepath or not ctx.triggered: # 파일 선택이 없거나, 앱 초기 로드 시에는 실행 안 함 (새로고침 버튼으로 유도)
        return no_update, no_update, no_update, no_update, 0, {}, 0, True, go.Figure(), "파일을 선택하거나 새로고침 버튼을 누르세요."

    try:
        status_messages = [f"파일 로딩 중: {os.path.basename(selected_filepath)}"]
        df_original = pd.read_csv(selected_filepath)

        internal_timestamp_col = 'Timestamp'
        if 'DATETIME' in df_original.columns:
            df_original.rename(columns={'DATETIME': internal_timestamp_col}, inplace=True)
        elif internal_timestamp_col not in df_original.columns:
            return no_update, no_update, no_update, no_update, 0, {}, 0, True, go.Figure(), f"오류: '{internal_timestamp_col}' 또는 'DATETIME' 컬럼이 없습니다."

        df_original[internal_timestamp_col] = pd.to_datetime(df_original[internal_timestamp_col])
        df_indexed = df_original.set_index(internal_timestamp_col)

        status_messages.append(f"원본 데이터 포인트 수: {len(df_indexed)}")

        numeric_cols = df_indexed.select_dtypes(include=np.number).columns
        df_resampled = pd.DataFrame()
        if not numeric_cols.empty:
            valid_numeric_cols = [col for col in numeric_cols if col in df_indexed.columns]
            if valid_numeric_cols:
                df_resampled = df_indexed[valid_numeric_cols].resample('1H').first()
                df_resampled.dropna(how='all', inplace=True)
                status_messages.append(f"1시간 단위 리샘플링 완료. (결과: {len(df_resampled)}개)")
            else:
                df_resampled = df_indexed.copy()
                status_messages.append("경고: 유효 숫자형 컬럼 부재로 리샘플링 건너뜀.")
        else:
            df_resampled = df_indexed.copy()
            status_messages.append("경고: 숫자형 컬럼 부재로 리샘플링 건너뜀.")

        if df_resampled.empty:
            return no_update, no_update, no_update, no_update, 0, {}, 0, True, go.Figure(), "\n".join(status_messages + ["리샘플링 후 데이터가 없습니다."])

        mapped_stages = map_columns_from_csv(df_resampled.columns)
        if not mapped_stages:
            return no_update, no_update, no_update, no_update, 0, {}, 0, True, go.Figure(), "\n".join(status_messages + ["스테이지 컬럼 매핑 실패."])

        all_temps, all_ws = [], []
        for _, row in df_resampled.iterrows():
            for stage in mapped_stages:
                try:
                    temp, rh = row[stage['temp_col']], row[stage['rh_col']]
                    if pd.isna(temp) or pd.isna(rh): continue
                    all_temps.append(temp)
                    w = get_humidity_ratio_kg_kg(temp, rh)
                    if not pd.isna(w) and w != float('inf') and 0 <= w < 0.1: all_ws.append(w)
                except KeyError: continue

        if not all_temps or not all_ws:
            min_temp_data, max_temp_data, min_w_data, max_w_data = 0, 40, 0, 0.025
        else:
            min_temp_data, max_temp_data = min(all_temps), max(all_temps)
            min_w_data, max_w_data = min(all_ws), max(all_ws)

        temp_margin = max(5, (max_temp_data - min_temp_data) * 0.1 if (max_temp_data - min_temp_data) > 0 else 5)
        w_margin = max(0.002, (max_w_data - min_w_data) * 0.1 if (max_w_data - min_w_data) > 0 else 0.002)
        chart_min_temp, chart_max_temp = min_temp_data - temp_margin, max_temp_data + temp_margin * 1.5
        chart_min_w, chart_max_w = 0, min(0.035, max_w_data + w_margin * 2.0)
        w_sat_at_max_temp = get_humidity_ratio_kg_kg(chart_max_temp, 100)
        if not pd.isna(w_sat_at_max_temp) and w_sat_at_max_temp != float('inf'):
            chart_max_w = max(chart_max_w, w_sat_at_max_temp * 1.05)

        chart_ranges = {'temp_range_c': (chart_min_temp, chart_max_temp), 'w_range_kg_kg': (chart_min_w, chart_max_w)}

        # 정적 배경 Figure 생성
        fig_background = create_psychrometric_background_figure(chart_ranges['temp_range_c'], chart_ranges['w_range_kg_kg'])
        static_traces_json = fig_background.to_json() # 정적 트레이스 저장

        # 초기 동적 데이터로 Figure 업데이트
        fig = go.Figure(data=fig_background.data) # 배경 트레이스로 초기화
        fig.add_trace(go.Scatter(x=[], y=[], mode='markers', name="공조 지점", showlegend=False, uid="dynamic-points"))
        fig.add_trace(go.Scatter(x=[], y=[], mode='lines', line=dict(color='lightgrey', width=2), name="공조 경로", showlegend=False, uid="dynamic-path"))

        initial_slider_value = 0
        row_data = df_resampled.iloc[initial_slider_value]
        timestamp = df_resampled.index[initial_slider_value]
        points_x, points_y, texts, customdata, colors, annotations = \
            get_dynamic_trace_data_for_timestamp(row_data, mapped_stages, chart_ranges)

        fig.data[-2].x = points_x
        fig.data[-2].y = points_y
        fig.data[-2].text = texts
        fig.data[-2].customdata = np.array(customdata).T if customdata and any(c is not None for c_list in customdata for c in c_list) else np.array([]).reshape(2,0)
        fig.data[-2].marker.color = colors
        fig.data[-1].x = points_x
        fig.data[-1].y = points_y

        fig.update_layout(
            annotations=annotations,
            title_text=f"대화형 습공기선도: {os.path.basename(selected_filepath)} - {timestamp.strftime('%Y-%m-%d %H:%M:%S')}",
            xaxis_range=chart_ranges['temp_range_c'],
            yaxis_range=chart_ranges['w_range_kg_kg'],
            width=1200, height=800,
            legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1, tracegroupgap=10),
            margin=dict(l=70, r=50, b=120, t=100), hovermode="closest",
            plot_bgcolor='rgba(245,245,245,1)'
        )

        slider_max = len(df_resampled) - 1
        slider_marks = {i: df_resampled.index[i].strftime('%m-%d %H시') for i in range(0, len(df_resampled), max(1, len(df_resampled)//12))}

        status_messages.append(f"로드 완료: {os.path.basename(selected_filepath)} ({len(df_resampled)}개 시간대)")
        if len(df_resampled) > 500: # 프레임이 너무 많으면 경고
             status_messages.append(f"주의: 시간대(프레임)가 {len(df_resampled)}개로 많아 반응이 느릴 수 있습니다.")


        return df_resampled.reset_index().to_json(date_format='iso', orient='split'), \
               mapped_stages, chart_ranges, static_traces_json, \
               slider_max, slider_marks, initial_slider_value, False, \
               fig, "\n".join(status_messages)

    except Exception as e:
        print(f"데이터 처리 중 오류: {e}")
        traceback.print_exc()
        return no_update, no_update, no_update, no_update, 0, {}, 0, True, go.Figure(), f"오류 발생: {e}"


def get_dynamic_trace_data_for_timestamp(row_data, mapped_stages, chart_ranges):
    points_x, points_y, texts, customdata, colors = [], [], [], [], []
    annotations = []
    chart_max_w = chart_ranges['w_range_kg_kg'][1]

    for stage_conf in mapped_stages:
        temp_c, rh_p = np.nan, np.nan
        try: temp_c, rh_p = row_data[stage_conf['temp_col']], row_data[stage_conf['rh_col']]
        except KeyError: pass

        w, h = np.nan, np.nan
        if not (pd.isna(temp_c) or pd.isna(rh_p)):
            w = get_humidity_ratio_kg_kg(temp_c, rh_p)
            h = get_enthalpy_kJ_kg(temp_c, rh_p)

        is_valid_point = not (pd.isna(w) or pd.isna(h) or w == float('inf') or h == float('inf') or w >= chart_max_w * 1.2 or w < chart_ranges['w_range_kg_kg'][0])

        if is_valid_point:
            points_x.append(temp_c); points_y.append(w); texts.append(stage_conf['name'])
            customdata.append([rh_p, h]); colors.append(stage_conf['color'])
            annotations.append(dict(
                x=temp_c, y=w, text=f"<b>{stage_conf['name']}</b><br>{temp_c:.1f}°C, {rh_p:.0f}%<br>h={h:.1f}kJ/kg",
                showarrow=True, arrowhead=1, arrowwidth=1, arrowcolor=stage_conf['color'], ax=20, ay=-30,
                font=dict(size=9), bordercolor=stage_conf['color'], borderwidth=1, bgcolor="rgba(255,255,255,0.85)" ))
        else:
            points_x.append(None); points_y.append(None); texts.append(stage_conf['name'] + " (데이터 없음)")
            customdata.append([np.nan, np.nan]); colors.append('rgba(200,200,200,0.5)')
    return points_x, points_y, texts, customdata, colors, annotations


@app.callback(
    Output('psychrometric-chart', 'figure',allow_duplicate=True),
    [Input('time-slider', 'value'),
     State('dataframe-store', 'data'),
     State('mapped-stages-store', 'data'),
     State('chart-ranges-store', 'data'),
     State('static-traces-store', 'data'), # 정적 트레이스 사용
     State('file-dropdown', 'value')],
    prevent_initial_call=True
)
def update_chart_on_slider(slider_value, df_json, mapped_stages, chart_ranges, static_traces_json, selected_filepath):
    if df_json is None or mapped_stages is None or chart_ranges is None or static_traces_json is None or selected_filepath is None:
        return no_update

    df_resampled = pd.read_json(df_json, orient='split', convert_dates=True)
    if 'Timestamp' not in df_resampled.columns and 'index' in df_resampled.columns: # reset_index()로 인해 'index' 컬럼이 생겼을 경우
        df_resampled.rename(columns={'index': 'Timestamp'}, inplace=True)

    if 'Timestamp' not in df_resampled.columns: # 여전히 Timestamp가 없으면 오류
        print("오류: 슬라이더 콜백에서 Timestamp 컬럼을 찾을 수 없습니다.")
        return no_update

    df_resampled.set_index('Timestamp', inplace=True)


    if slider_value >= len(df_resampled):
        return no_update

    row_data = df_resampled.iloc[slider_value]
    timestamp = df_resampled.index[slider_value]

    points_x, points_y, texts, customdata, colors, annotations = \
        get_dynamic_trace_data_for_timestamp(row_data, mapped_stages, chart_ranges)

    # 저장된 정적 배경 트레이스로 Figure 초기화
    fig = go.Figure(data=go.Figure(pd.read_json(static_traces_json, typ='frame', orient='split')).data)


    # 동적 트레이스 업데이트 또는 추가 (uid를 사용하여 기존 트레이스 업데이트 시도)
    # 주의: fig.data의 인덱스를 정확히 알아야 함.
    # 여기서는 배경 트레이스 이후에 동적 트레이스가 추가된다고 가정.
    # 더 안전한 방법은 uid로 트레이스를 식별하는 것.

    # 동적 트레이스 데이터 구성
    dynamic_points_trace = go.Scatter(
        x=points_x, y=points_y, text=texts,
        customdata=np.array(customdata).T if customdata and any(c is not None for c_list in customdata for c in c_list) else np.array([]).reshape(2,0),
        mode='markers', marker=dict(color=colors, size=10, line=dict(width=1, color='DarkSlateGrey')),
        name="공조 지점", showlegend=False, uid="dynamic-points"
    )
    dynamic_path_trace = go.Scatter(
        x=points_x, y=points_y, mode='lines', line=dict(color='lightgrey', width=2),
        name="공조 경로", showlegend=False, uid="dynamic-path"
    )

    # 기존 동적 트레이스를 업데이트하거나, 없으면 새로 추가 (더 강건한 방식)
    updated_data = list(fig.data) # 기존 배경 트레이스 복사

    point_trace_found = False
    path_trace_found = False

    for i, trace in enumerate(updated_data):
        if trace.uid == "dynamic-points":
            updated_data[i] = dynamic_points_trace
            point_trace_found = True
        elif trace.uid == "dynamic-path":
            updated_data[i] = dynamic_path_trace
            path_trace_found = True

    if not point_trace_found:
        updated_data.append(dynamic_points_trace)
    if not path_trace_found:
        updated_data.append(dynamic_path_trace)

    fig.data = updated_data


    fig.update_layout(
        title_text=f"대화형 습공기선도: {os.path.basename(selected_filepath)} - {timestamp.strftime('%Y-%m-%d %H:%M:%S')}",
        annotations=annotations,
        xaxis_range=chart_ranges['temp_range_c'],
        yaxis_range=chart_ranges['w_range_kg_kg'],
        width=1200, height=800,
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1, tracegroupgap=10),
        margin=dict(l=70, r=50, b=120, t=100), hovermode="closest",
        plot_bgcolor='rgba(245,245,245,1)'
    )
    return fig


# --- 실행 부분 ---
if __name__ == '__main__':
    if not os.path.isdir(DATA_DIRECTORY):
        print(f"경고: 데이터 디렉토리 '{DATA_DIRECTORY}'가 존재하지 않습니다. 스크립트와 같은 위치에 생성해주세요.")
        try:
            os.makedirs(DATA_DIRECTORY)
            print(f"'{DATA_DIRECTORY}' 디렉토리를 생성했습니다. 분석할 CSV 파일을 넣어주세요.")
        except OSError as e:
            print(f"디렉토리 생성 실패: {e}")

    # JupyterLab/Notebook 환경에서 실행 시 jupyter_mode 사용
    if 'JUPYTER_SERVER_REQUEST' in os.environ or 'JPY_PARENT_PID' in os.environ : # Jupyter 환경 감지
        print("Jupyter 환경에서 실행 중입니다. Dash 앱은 새 탭 또는 인라인으로 표시될 수 있습니다.")
        app.run_server(jupyter_mode="tab", debug=True, port=8051) # jupyter_mode 와 포트 지정
    else: # 일반 Python 스크립트로 실행 시
        app.run_server(debug=True)